In [1]:
%matplotlib inline  
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import numpy as np
import pandas as pd

from access_science_shared import standardizer
from access_literature_data import medline

sys.path.append('./../../src')

import resci_inout as inout
import nar170604f_occurences as nar_attention
import nar170823f_prediction_datasets as pred
import nar170830f_predictions as pred_manager

# Shared settings

In [3]:
project_base = '170923f_predict_human_fame_within_subcategories'
date_stamp = '170923'

In [4]:
taxon_id = 9606
ref_genes = standardizer.reference_genes(taxon_id, 'orp')

gene2pubmed = medline.gene2pubmed(taxon_id, paper_kind='research', ref_genes = ref_genes)
df_targets = nar_attention.count_papers_and_attention(ref_genes, gene2pubmed)

target_normalizer = lambda x: np.log10(x)

def maker(sub_name, u_features, df_target, target_normalizer):
    pred_manager.make_base(
        project_base = project_base,
        sub_name = sub_name,
        ref_genes = ref_genes,
        u_features = u_features,
        df_target = df_target.applymap(target_normalizer))
    
def get_u(list_of_categories):
    u = {}
    for x in list_of_categories:
        u.update(features[x])
    return u

def supermaker(of_interest, feature_to_predict):
    sub_name = '{}_human_{}_log_{}'.format(date_stamp, ''.join(of_interest), feature_to_predict)
    u_all = get_u(of_interest)
    maker(sub_name, u_all, df_targets[[feature_to_predict]], target_normalizer)

# Specific datasets

In [5]:
%%time
pool = pred.retreive_single_sets(ref_genes, taxon_id)

InputTerm has 2 ambiguous entries
InputTerm has 2 ambiguous entries
InputTerm has 2 ambiguous entries
InputTerm has 2 ambiguous entries
InputTerm has 2 ambiguous entries
InputTerm has 2 ambiguous entries
CPU times: user 4min 42s, sys: 52.9 s, total: 5min 35s
Wall time: 5min 43s


In [6]:
# create organization that mimicks individual sets
features = dict()
for k in pool.keys():
    d = pool[k]
    flat = {k: d}
    features[k] = flat

In [7]:
%%time
for feature_to_predict in ['attention', 'papers']:
    
    for k in pool.keys():
            
        supermaker([
            k       
        ], feature_to_predict)


CPU times: user 4min 49s, sys: 5.84 s, total: 4min 55s
Wall time: 4min 56s


In [8]:
print('done')

done
